# Hash Tables

## Dictionaries

A dictionrary stores key element pairs called items, allowing duplicates of keys.

- `findElement(k)`: If the dictionary contains k return it, otherwise return NO_SUCH_KEY
- `insertItem(k, e)`: Inserts an item with key k and element e
- `removeItem(k)`: If the dictionary contains k, remove it are returns its element, otherwise return NO_SUCH_KEY

A obviouse realization would be a singly linked list. insertItem would be $\Theta(1)$, since its equivalent to inserting a new head node, findElement and removeElement would be $\Theta(n)$ since the element with key k may be at the end of the linked list, thus n elements must be transversed.

If all keys where unique, integers, and the range of keys was $0$ to $N-1$ then the dictionary could be implemented with an array of length $N$. By using an array we get direct adressing thus all methods are $\Theta(1)$ (no need to transverse all elements). The downside of an array based implentation (except from the key restriction) is that if the number of elements is much less than $N$ we waste alot of space for a sparsly populated array.

The ADT for a dictionary in C++ would be as follows:

In [1]:
class HashMap {
    public:
        int findElement(std::string key);
        void insertItem(std::string key, int value);
        int removeItem(std::string key);
}

We are using strings and ints for simplisity, but you can imagine a `HashMap<K, V>`  where `K` is the key type and `V` is the value type.

## Bucket Arrays

If we allow duplicate keys and instead of storing elements at each index, we store list of all the elements with the same key we get bucket arrays. If we use a singly linked list for the bucket array, insertItem and removeItem become $O(1)$ ($O(1)$ for the array access and $O(1)$ for inserting/removing at head).

## Hash Tables

Hash tables are extension on bucket arrays allowing arbitary keys. A hash table is backed by a bucket array of length $B$ of length $N$ with dictionarys in each bucket. The keys are mapped to the range $0, ..., N-1$ by a hash function, after computing the hash we insert the item into the dictionary at `B[h(k)]`.

The run time of findElement, insertItem and removeItem depends on the runtime of h, let $T_h$ be the worst case runtime for h. The runtime of h may depend on key size, but key size is often constant, thus we wont consider it here. Now let us consider findElement(k), which corresponds to `B[h(k)].findElement(k)`. The runtime of findElement is thus $T_h + O(m)$ where $m$ is the size of the dictionary at `B[h(k)]`. By the same reasoning removeElement is also $T_h + O(m)$.

In the worst case are hash function sends all items to the same key and $m = n$, thus a hash table is no more efficent than a dictionary, however with the appropriate hash function and uniform keys we can expect $m = 1$, thus the runtime of all methods becomes $T_h + O(1)$.

## Hash Functions

A good hash function needs two properties:

1. h evenly distributes keys over the whole range (thus the dictionarys are approximatly the same size)
2. h is easy to compute

A hash function needs to convert an arbitary key into an interger value, before compressing it into the range of $1, ..., N - 1$. The most obvious method is to consider the bit representation of key, however this may not be of fixed size. We could only consider the first $k$ bits, however all keys with the same first $k$ bits will have the same key, thus we need to use all of the key.

- __Sumation hash code:__ Consider the bit representation of the key as a sequence of interger values $a_0, a_1, ..., a_l$. A summartion hash code is the sum of $a_0$ to $a_l$, thus using all the bits of the key. One problem is their are regonizable patterns to the way keys are assigned hash codes, for example $a_0, a_1, a_2$ and $a_1, a_0, a_2$ and $a_2, a_0, a_1$ will all get mapped to the same key.
- __Polynomial hash code:__ Instead of just adding $a_0, ... a_l$ we can choose a fixed $x$, and let the hash code correspond to $a_0 + a_1 x + a_2 x^2, ..., a_l x^{l}$. This approach leads to much more even distribution of keys, when $x$ is coprime with $N$. A nieve implemtation would require $O(l^2)$ additions and multiplications to compute the code, however if we rewrite $a_0 + a_1 x + a_2 x^2 + ... + a_l x^{l}$ using Horner's rule we get $a_0 + x(a_1 + x(a_2 + ... + x(a_{l_1} + x a_l)...))$ which can be evaluated with $O(l)$ operations.

Many hash functions dont have a compression phase, however it can be usefull in some domains to compute a larger hash code, then reduce it. In this case a good compression function is:

$$
|ak + b| \mod N
$$

where $a$ and $b$ are random number choosen at runtime (when the hashtable is created). This works well when $a$ and $N$ are coprime, however if they are not then the mapping is bad.

Lets assume $a$, $b$ and $k$ are non-negative. A good hash function should hash atleast one key to each location $r$ in the array, that is:

$$
ak + b \equiv r \mod N
$$

sould have a soultion for each integer $k$. By defintion of mod we get:

$$
ak + b = qN + r
$$

for some integers $q$ and $r$. Rearranging:

$$
ak - qN = r - b
$$

thus if their is some $d$ that divides $a$ and $N$, then it divides $r - b$ thus $r - b = sd$. Thus $r = b + sd$ for some integers $s$, thus the only locations we can hash are those in the form $b + sd$.

Here is an example of a polynomial hash function:

In [2]:
unsigned hash(const char* s, unsigned salt)
{
    unsigned h = salt;
    while (*s)
        h = h * 101 + (unsigned) *s++;
    return h;
}

In the examples bellow `foo`, `bar` and `baz` have diffrent hashes virtually every time

In [3]:
#include <iostream>

unsigned SALT = std::rand();
std::cout << "foo: " << hash("foo", SALT) << std::endl;
std::cout << "bar: " << hash("bar", SALT) << std::endl;
std::cout << "baz: " << hash("baz", SALT) << std::endl;

foo: 781684339
bar: 781642124
baz: 781642132


## Implementation

We will give the table some arbitary size, called `TABLE_SIZE`.

In [4]:
const int TABLE_SIZE = 128;

The dictionarys at each entry in the bucket array will be `std::map` in this case. While they are not quite dictionarys, they will be sufficent to demo.

In [5]:
typedef std::map<std::string, int> TableEntry;
TableEntry table[TABLE_SIZE];

`findElement` uses the hash function to get the index into the bucket array. `at` finds a key in the `std::map`, throwing an exception if it doesnt exist.

In [6]:
int HashMap::findElement(std::string key) {
    int index = hash(key.c_str(), SALT) % TABLE_SIZE;
    return table[index].at(key);
}

`insertItem` is simular to `findElement` except it inserts a `key` `value` pair into corresponding `std::map`

In [7]:
void HashMap::insertItem(std::string key, int value) {
    int index = hash(key.c_str(), SALT) % TABLE_SIZE;
    table[index].insert(std::make_pair(key, value));
}

Finnaly an example of the hashmap in use:

In [8]:
HashMap map;
map.insertItem("foo", 10);
map.insertItem("bar", 20);
map.insertItem("baz", 30);
std::cout << "foo = " << map.findElement("foo") << std::endl;

foo = 10


## Load factor
The load factor of a hash table is $n / N$, the approximate size of the dictionarys given the keys are evenly distributed. A common load factor is $3/4$. A good chooise for $N$ is a prime (of size roughly $(4/3)n$, if $N$ is prime, it will be coprime for all $a < 2N$ (so long as $N \ne 2$).

As items are added/removed from the hashtable, if the load factor is ever to low or high, the hash table must be re-hashed, that is a new hashtable is created and all the items are copyed over (using the new hash function).